In [ ]:
!nvidia-smi

In [ ]:
# Install dependencies
!pip install roboflow
!pip install torch torchvision
!pip install gdown

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9
%cd yolov9

In [ ]:
!pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

weights_dir = "/content/drive/MyDrive/yolov9_weights"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="CjO2MQmvyPJDsxvgneCq")
project = rf.workspace("my-project-xhvrg").project("skin-disease-ges3o")
version = project.version(5)
dataset = version.download("yolov9")

In [ ]:
!ls {dataset.location}

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class YOLOv9Custom(nn.Module):
    def __init__(self):
        super(YOLOv9Custom, self).__init__()
        # Insert YOLOv9 layers
        # Modify by adding ReLU activation in appropriate layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        # Add more YOLOv9 layers here with ReLU

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        # Forward pass through other YOLOv9 layers
        return x

In [ ]:
model = YOLOv9Custom()

In [ ]:
weights_path = "path/to/custom/weights.pt"
model.load_state_dict(torch.load(weights_path, map_location='cpu'))

In [ ]:
from yolov9.train import train

# Configure the training settings
train_cfg = {
    'epochs': 50,
    'batch_size': 16,
    'data': f'{dataset.location}/data.yaml',
    'weights': weights_path,  #custom weights
    'save_dir': weights_dir,  # Save model weights in gdrive
    'img_size': 640,
    'optimizer': 'SGD',
}

In [ ]:
train(train_cfg)

In [ ]:
torch.save(model.state_dict(), f'{weights_dir}/best_yolov9_weights.pt')

In [ ]:
from yolov9.val import val

In [ ]:
val_cfg = {
    'data': f'{dataset.location}/data.yaml',
    'weights': f'{weights_dir}/best_yolov9_weights.pt',
    'img_size': 640,
}

val(val_cfg)

In [ ]:
# Convert PyTorch model to TensorFlow .h5 format
import tensorflow as tf

# Save the trained PyTorch model as .h5 (simplified conversion)
dummy_input = torch.randn(1, 3, 640, 640)  # A sample input shape
torch.onnx.export(model, dummy_input, "yolov9.onnx")

# Convert the ONNX model to TensorFlow format
!pip install onnx onnx-tf
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("yolov9.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("yolov9_model.h5")

# Save .h5 in Google Drive
!mv yolov9_model.h5 /content/drive/MyDrive/yolov9_model.h5
